                                           Reviews-Bangalore

In [113]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from selenium.common.exceptions import NoSuchElementException
import sys

In [2]:
header={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.117 Safari/537.36'}

## Function to get the webpage

In [3]:
def Search(search_query):
    url='https://www.practo.com/search?results_type=doctor&q=%5B%7B%22word%22%3A%22general%20physician%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city=Bangalore&page='+search_query
    time.sleep(1)
    page=requests.get(url,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

## Function to scrape the link of individual doctor's profile

In [4]:
link=[]
for i in range(95):  #Doctor's profiles are in 95 pages
    search_response=Search(str(i))
    soup=BeautifulSoup(search_response.content) #soup has the content of the page
    links= soup.findAll('a',{'class':"u-color--primary"}) #accessing the link of the doctor's profile
    for l in links:
        link.append(l.get('href')) #saves all the links

In [5]:
df=pd.DataFrame()

df['link']=link

## Function to change the address of the link(so that we will get the recent reviews)

In [6]:
def clean(row):
    Link=row['link']
    Link=Link.split('?')
    return Link[0]+'/recent'

In [7]:
df['link']=df.apply(lambda row:clean(row),axis=1) #calling the function

In [12]:
df=df.drop_duplicates(subset='link').reset_index(drop=True) #droping the duplicates links

In [13]:
Links=df['link']

## Initializing selenium 

###### As we have infinite scrolling, selenium will be easy to click the more button

In [183]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(executable_path='C:/Users/VIJESH\Desktop/chromedriver.exe')

## Function to click more button (to get all the reviews of the respective doctor)

In [184]:
def review_page(query):    
    driver.get("https://www.practo.com"+Links[query]) #opens the doctor's profile
    for i in range(45):                               #45 times it clicks the more button
        try:
            driver.find_element_by_xpath('//button[@data-qa-id="view-more-feedback"]').click()
            time.sleep(1)
        except NoSuchElementException:
            pass
    return driver.page_source

## Function to scrape the reviews

In [185]:
review=[]
rev=''
recom=[]
import time
for i in range(len(Links)):
    soup = BeautifulSoup(review_page(int(i)), 'lxml')   #soup has the content of the doctor's profile
    for rev_clas in soup.findAll(attrs={'class':"feedback__body"}): #acessing the tag in which reviews are present
        for i in rev_clas.findAll(attrs={'data-qa-id':"review-text"}):#saves all the patient reviews 
            rev=rev+i.text
        review.append(rev)
        rev=''
    for rev_clas in soup.findAll(attrs={'class':"pure-g feedback--item u-cushion--medium-vertical"}):
        for i in rev_clas.findAll(attrs={'class':'u-cushion--small-bottom u-large-font'}):#saves the recommendations
            recom.append(i.text)

In [186]:
len(review) #length of the reviews

9277

In [187]:
df_1=pd.DataFrame()

In [188]:
df_1['reviews']=review

In [189]:
df_1=df_1.drop_duplicates() #dropping all duplicates

## Function to tag the location

In [193]:
def location(row):
    each_row=row['reviews']
    return 'Bangalore'

In [194]:
df_1['location']=df_1.apply(lambda row:location(row),axis=1) #calling function

In [195]:
df_1.head()

,reviews,location
0,This doctor is so to the point and focused and...,Bangalore
1,*** ** *** **** ****** ** ********** ***** She...,Bangalore
2,Doctor listened very patiently and explained ...,Bangalore
3,such a wonderfull doctor who explains clearly ...,Bangalore
4,Even after booking through practo we had to w...,Bangalore


In [196]:
df_1.to_csv('doctor_Bangalore.csv',index=False) #saving the dataframe as CSV